In [54]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [55]:
index_name = "course-questions-hw"

In [56]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

# Delete the index if it exists
if es_client.indices.exists(index=index_name):
    es_client.indices.delete(index=index_name)
    print(f"Deleted index: {index_name}")

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

es_client.indices.create(index=index_name, body=index_settings)


Deleted index: course-questions-hw


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-hw'})

In [57]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, body=doc)

es_client.indices.refresh(index=index_name)

  0%|          | 0/948 [00:00<?, ?it/s]

ObjectApiResponse({'_shards': {'total': 1, 'successful': 1, 'failed': 0}})

In [58]:
q = "How to copy a file to a Docker container?"

In [59]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name , body=search_query)   

    result_docs = []

    for hit in response['hits']['hits']:
        doc = hit['_source']
        doc['_score'] = hit['_score']  # Add the score to the document
        result_docs.append(doc)

    return result_docs

In [60]:
elastic_search(q)

[{'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
  'course': 'machine-learning-zoomcamp',
  '_score': 60.810158},
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug 

In [61]:
def build_prompt(question):
    text = elastic_search(question)

    context_template = """
    Q: {question}
    A: {text}
    """.strip()

    context = context_template.format(question=question, text=text)

    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()

    prompt = prompt_template.format(question=question, context=context)

    return prompt

In [62]:
prompt = build_prompt(q)

len(prompt)

1858

In [63]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: How to copy a file to a Docker container?

    CONTEXT:
    Q: How to copy a file to a Docker container?
    A: [{'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani", 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from my local machine to docker container?', 'course': 'machine-learning-zoomcamp', '_score': 60.810158}, {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker r

In [64]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")   

# Count tokens
tokens = encoding.encode(prompt)
num_tokens = len(tokens)
num_tokens

446